In [662]:
import openpyxl as pxl

In [663]:
import pandas as pd

In [664]:
from pathlib import Path

In [665]:
input_file = Path.cwd().joinpath("input").joinpath("logs")

In [666]:
input_file

WindowsPath('C:/Users/narayanan/Downloads/hw22_visweswaran/hw22_visweswaran/notebooks/input/logs')

In [667]:
for f in input_file.rglob('BCM*.csv'):
    print(f"filename = {f.name}, stem = {f.stem}, suffix = {f.suffix}")

filename = BCM-E-tCenter-Deep.csv, stem = BCM-E-tCenter-Deep, suffix = .csv
filename = BCM-E-tCenter-Medium.csv, stem = BCM-E-tCenter-Medium, suffix = .csv
filename = BCM-E-tCenter-Shallow.csv, stem = BCM-E-tCenter-Shallow, suffix = .csv
filename = BCM-E-tLeft-Deep.csv, stem = BCM-E-tLeft-Deep, suffix = .csv
filename = BCM-E-tLeft-Medium.csv, stem = BCM-E-tLeft-Medium, suffix = .csv
filename = BCM-E-tRight-Deep.csv, stem = BCM-E-tRight-Deep, suffix = .csv
filename = BCM-E-tRight-Medium.csv, stem = BCM-E-tRight-Medium, suffix = .csv
filename = BCM-N-tCenter-Deep.csv, stem = BCM-N-tCenter-Deep, suffix = .csv
filename = BCM-N-tCenter-Medium.csv, stem = BCM-N-tCenter-Medium, suffix = .csv
filename = BCM-N-tLeft-Deep.csv, stem = BCM-N-tLeft-Deep, suffix = .csv
filename = BCM-N-tLeft-Medium.csv, stem = BCM-N-tLeft-Medium, suffix = .csv
filename = BCM-N-tRight-Deep.csv, stem = BCM-N-tRight-Deep, suffix = .csv
filename = BCM-N-tRight-Medium.csv, stem = BCM-N-tRight-Medium, suffix = .csv
filena

In [668]:
dfs={}

In [669]:
for f in input_file.rglob('BCM*.csv'):
    fstem = f.stem
    df = pd.read_csv(f,header=None)
    dfs[fstem] = df

In [670]:
with pd.ExcelWriter('output/BCM.xlsx') as writer:
    for sheetname,sheetdata in dfs.items():
            sheetdata = sheetdata.astype({0: 'datetime64',1: 'string',2:'float64'})
            sheetdata.to_excel(writer,sheet_name=sheetname,index=False,header=['datetime','scale','temperature'])

In [671]:
wb = pxl.load_workbook("output/BCM.xlsx")

In [672]:
from statistics import mean

In [673]:
for x in wb.sheetnames:
    ws = wb[x]
    ws['G2'] = "min_temp"
    ws['G3'] = "max_temp"
    ws['G4'] = "mean_temp"
    ws['G6'] = "min_date"
    ws['G7'] = "max_date"
    date_cells = []
    temperature_cells = []
    for row_pack in ws['A2':'A'+str(ws.max_row)]:
        for row in row_pack:
            date_cells.append(row.value)
    ws['H6'] = min(date_cells)
    ws['H7'] = max(date_cells)
    for row_pack in ws['C2':'C'+str(ws.max_row)]:
        for row in row_pack:
            temperature_cells.append(row.value)
    ws['H2'] = min(temperature_cells)
    ws['H3'] = max(temperature_cells)
    ws['H4'] = mean(temperature_cells)
wb.save('output/BCM.xlsx')   

Hacker


In [674]:
set_of_files = set()
dire = Path('input/logs_hacker')
dfs_hacker = {}
for f in dire.glob('*.csv'):
    fstem = f.stem
    df_hacker = pd.read_csv(f,header=None)
    dfs_hacker[fstem] = df_hacker
    set_of_files.add(fstem[0:fstem.index('-')])
    

In [675]:
for x in list(set_of_files):
    wb = pxl.Workbook()
    wb.save('output_hacker/'+x+'.xlsx')
    with pd.ExcelWriter('output_hacker/'+x+'.xlsx') as writers:
      for sheetname,sheetdata in dfs_hacker.items():
        if sheetname[0:sheetname.index('-')] == x:
            sheetdata = sheetdata.astype({0: 'datetime64',1: 'string',2:'float64'})
            sheetdata.to_excel(writers,sheet_name=sheetname,index=False,header=['datetime','scale','temperature'])
        else:
            continue
   
    
    